In [1]:
# ====================================================
# 🚀 Real-ESRGAN Colab Setup | PyTorch 2.x Compatible
# Fixed model initialization
# ====================================================

# 1. 📦 Install dependencies
!pip install --upgrade torch torchvision torchaudio --quiet
!pip install realesrgan basicsr gfpgan Pillow numpy --quiet

# 2. 🩹 Patch import issue in basicsr for PyTorch ≥ 2.0
!sed -i 's#from torchvision.transforms.functional_tensor import rgb_to_grayscale#from torchvision.transforms.functional import rgb_to_grayscale#g' /usr/local/lib/python3.*/dist-packages/basicsr/data/degradations.py

# 3. 📥 Download Real-ESRGAN pretrained model weights
!wget -q https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -O RealESRGAN_x4plus.pth

# (Optional: for anime)
# !wget -q https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/RealESRGAN_x4plus_anime_6B.pth -O RealESRGAN_x4plus_anime_6B.pth

# 4. 🧠 Initialize model properly
import torch
import numpy as np
from PIL import Image
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet  # <-- this is important!

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define the RRDBNet architecture used in RealESRGAN_x4plus
model_arch = RRDBNet(
    num_in_ch=3,
    num_out_ch=3,
    num_feat=64,
    num_block=23,
    num_grow_ch=32,
    scale=4
)

# Create RealESRGANer with actual model object
model = RealESRGANer(
    scale=4,
    model_path='RealESRGAN_x4plus.pth',
    model=model_arch,
    device=device
)

print(f"✅ Real-ESRGAN model loaded successfully on {device.upper()}!")

# ====================================================
# 🖼️ Helper functions for image upscaling
# ====================================================
import os

def upscale_image(input_path: str, output_path: str):
    """Enhance a single image and save the output."""
    img = Image.open(input_path).convert('RGB')
    img_np = np.array(img)
    output, _ = model.enhance(img_np)
    Image.fromarray(output).save(output_path)
    print(f"✅ Upscaled image saved at: {output_path}")

def upscale_folder(input_folder: str, output_folder: str):
    """Batch upscale all images in a folder."""
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            in_path = os.path.join(input_folder, filename)
            out_path = os.path.join(output_folder, f"upscaled_{filename}")
            upscale_image(in_path, out_path)

# ====================================================
# 📤 Usage Example
# ====================================================
# 🖼️ Single image:
# upscale_image('your_image.png', 'upscaled_your_image.png')

# 🗂️ Batch images:
# upscale_folder('input_images', 'upscaled_results')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.5 MB/s eta 0:00:00
✅ Real-ESRGAN model loaded successfully on CUDA!


In [3]:
upscale_image('your_image.jpg', 'upscaled_your_image.jpg')

✅ Upscaled image saved at: upscaled_your_image.jpg
